In [11]:
from datetime import datetime
from pathlib import Path

import pandas as pd

In [23]:
data_path = Path("../data")

columns_of_interest = ["Datum", "Typ", "Wert", "Buchungswährung"]
how_many_types_expected = 5

type_mapping = {
    # Map Mintos types to Portfolio Performance types
    "Tilgungszahlungen": None,
    "Zinszahlungen": "Zinsen",
    "Erhaltene Zinsen aus Kreditrückkauf": "Zinsen",
    "Zinseinnahmen aus Kreditrückkauf": "Zinsen",
    "Erhaltene Zinsen": "Zinsen",
    "Erhaltene Tilgung aus Kreditrückkauf": None,
    "Übertragungsabgleich der Verzugszinseinnahmen": "Zinsen",
    "Investitionen in Darlehen": None,
    "Investition": None,
    "Zinseinnahmen aus ausstehenden Zahlungen": "Zinsen",
    "Steuereinbehalt": "Steuern",
    "Erhaltene Tilgung aus Rückkauf kleiner Kreditteile": None,
    "Erhaltene Tilgung": None,
    "Zweitmarkt-Transaktion – Ab- oder Aufschlag": "Zinsen",  # noqa: RUF001
    "Zweitmarkt-Transaktion": None,
    "Zweitmarktgebühr": "Gebühren",
    "Einzahlungen": "Einlage",
    "Abhebung": "Entnahme",
    "Antrag auf Beitrag/Spende": "Gebühren",
}

In [24]:
statement_files = list(data_path.glob("*.csv"))

[x.name for x in statement_files]

['20240508-account-statement.csv',
 '20240809-account-statement.csv',
 '2020-account-statement.csv',
 '20241114-account-statement.csv',
 '2022-account-statement.csv',
 '20240611-account-statement.csv',
 '2021-account-statement.csv',
 '2023-account-statement.csv']

In [25]:
data = (
    pd.concat([pd.read_csv(x) for x in list(data_path.glob("*.csv"))])
    .convert_dtypes()
    .assign(
        Datum=lambda df: pd.to_datetime(df["Datum"]).dt.date,
        Typ=lambda df: df["Zahlungsart"].replace(type_mapping),
        Wert=lambda df: df.groupby([df["Datum"], "Typ"])["Umsatz"].transform("sum").round(5),
    )
    .rename(columns={"Währung": "Buchungswährung"})
    .dropna(subset="Typ")
    .drop_duplicates(columns_of_interest)
)

In [26]:
if len(data["Typ"].unique().tolist()) != how_many_types_expected:
    raise Exception(f"Expecting {how_many_types_expected} distinct values here..")


In [27]:
data["Typ"].unique().tolist()

['Zinsen', 'Steuern', 'Entnahme', 'Einlage', 'Gebühren']

In [17]:
output_file_name = data_path / "output" / f"{datetime.now().strftime('%Y-%m-%d')}.csv"

if not output_file_name.parent.exists():
    output_file_name.parent.mkdir()


data[columns_of_interest].to_csv(
    output_file_name,
    index=False,
    sep=";",
    decimal=",",
)